In [17]:
using JuMP, GLPK
using DataFrames, CSV, XLSX

In [18]:
# PARAMETERS
F = 7       # number of fish types
G = 16       # number of fishing grounds
I = 53       # number of processing facilities
J = 40     # number of cities

# DATA: FISH 
df_fish = DataFrame(CSV.File("fish_data.csv"))
fish_price = df_fish."2021 Market Price"
fish_health_value = df_fish."Protein Value"

# DATA: FISHING GROUND
df_fishingground_max = DataFrame(CSV.File("fishingground_restriction.csv"))

# DATA: PROCESSING FACILITY
df_processingfacility = DataFrame(CSV.File("processing_facility.csv"))
processing_cap = df_processingfacility."Processing Capacity"
operational_cost = df_processingfacility."Operational Cost"

# DATA: CITIES
df_cities = DataFrame(CSV.File("cities_data.csv"))
cities_budget = df_cities."Average Yearly Budget of a Person to buy Fish"
cities_population = df_cities."Population (what year?)"
cities_min_consumption = df_cities."Minimum Fish Consumption"
cities_max_consumption = df_cities."Maximum Fish Consumption"

# DATA FOR OBJECTIVE FUNCTION
fishing_cost_3d = [
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 7")...)
]
transportation_cost_3d = [
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 7")...)
]

7-element Vector{DataFrame}:
 53×100 DataFrame
 Row │ 1        2        3        4        5        6        7        8        ⋯
     │ Any      Any      Any      Any      Any      Any      Any      Any      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 5.27071  8.72569  722.814  817.425  1.82021  318.987  472.962  21.0398  ⋯
   2 │ 477.028  477.762  354.176  336.004  481.554  736.815  13.2156  466.454
   3 │ 9.7982   12.1435  709.861  802.578  14.1971  328.518  458.088  12.6319
   4 │ 13.5411  11.3988  711.857  801.814  16.3915  324.595  457.229  5.94622
   5 │ 652.188  653.301  278.171  160.175  656.819  912.69   185.206  642.286  ⋯
   6 │ 11.2093  11.3621  728.861  823.1    6.33721  313.494  478.599  24.3848
   7 │ 8.63362  14.5472  709.208  804.318  13.6842  331.104  459.939  18.3479
   8 │ 13.1959  20.3829  705.998  803.645  17.8962  336.129  459.431  24.6925
   9 │ 1.73936  8.80826  718.929  813.707  4.21076  322.456  469.267  19.4037  ⋯

In [19]:
# VARIABLES
m = Model(GLPK.Optimizer)
@variable(m, x[1:F, 1:G, 1:I])
@variable(m, y[1:F, 1:I, 1:J])
@variable(m, z[1:I], Bin)
@variable(m, s[1:J])

40-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[29]
 s[30]
 s[31]
 s[32]
 s[33]
 s[34]
 s[35]
 s[36]
 s[37]
 s[38]
 s[39]
 s[40]

In [20]:
# CONSTRAINTS
# Non-negativity constraints
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f, g, i] >= 0)
        end
    end
end
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f, i, j] >= 0)
        end
    end
end
for j in 1:J
    @constraint(m, s[j] >= 0)
end      

# Fishing Restriction Constraint
for f in 1:F
    for g in 1:G
        @constraint(m, sum(x[f, g, i] for i in 1:I) <= df_fishingground_max[g, f])
    end
end

# Processing Capacity Constraint
for i in 1:I
    @constraint(m, sum(y[f, i, j] for f in 1:F for j in 1:J) <= processing_cap[i])
end

# Protein Consumption Constraint 
for j in 1:J 
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) <= 43800 * cities_population[j])
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) >= 17520 * cities_population[j])
end

# Distribution Capacity Constraint
@constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) <= sum(x[f, g, i] for f in 1:F for g in 1:G for i in 1:I))

# # Facility Opening Constraint
# # Part 1
# for i in 1:I
#     @constraint(m, sum(x[f,g,i] for f in 1:F for g in 1:G) <= sum(z[i] * df_fishingground_max[g, f] for f in 1:F for g in 1:G) )
# end
# # Part 2
# for i in 1:I
#     @constraint(m, sum(y[f,i,j] for f in 1:F for j in 1:J) <= z[i] * processing_cap[i])
# end


# Indicator variables
# If we close facility i, we cannot send any fish to facility i
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f,g,i] <= z[i] * df_fishingground_max[g, f])
        end
    end
end

# Indicator variables
# If we close facility i, we cannot send any fish from facility i
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f,i,j] <= z[i] * processing_cap[i])
        end
    end
end



# Budget Constraint
for j in 1:J
    @constraint(m, sum(y[f, i, j] * fish_price[f] for f in 1:F for i in 1:I)  <= cities_population[j] * (0.15) * cities_budget[j] + s[j])
end

In [21]:
# OBJECTIVE FUNCTION
@objective(m, Max, sum(fish_health_value[f]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(fishing_cost_3d[f][g, i]*x[f, g, i] for f in 1:F for g in 1:G for i in 1:I) - sum(transportation_cost_3d[f][i,j]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(operational_cost[i]*z[i] for i in 1:I) - sum(s[j] for j in 1:J))

258.25929424568847 y[1,1,1] + 254.80430663578795 y[1,1,2] - 459.28396811113544 y[1,1,3] - 553.8952172696688 y[1,1,4] + 261.70978673089274 y[1,1,5] - 55.45670913709313 y[1,1,6] - 209.43201477204093 y[1,1,7] + 242.49023400130218 y[1,1,8] + 248.8171441431085 y[1,1,9] + 249.70850333614237 y[1,1,10] - 397.4735116445162 y[1,1,11] + 257.89499840006386 y[1,1,12] + 234.27488653823963 y[1,1,13] - 612.7579946050911 y[1,1,14] + 249.57153191852214 y[1,1,15] + 246.47592137569217 y[1,1,16] + 260.4170603582854 y[1,1,17] + 245.85861406942627 y[1,1,18] + 253.0657580008916 y[1,1,19] + 245.18186610437425 y[1,1,20] - 145.3270683710242 y[1,1,21] - 127.64331143678015 y[1,1,22] + 237.33398918966614 y[1,1,23] + 219.47575076796286 y[1,1,24] - 219.80939000103785 y[1,1,25] + 243.4678893134426 y[1,1,26] + 206.8999709999523 y[1,1,27] + 250.1120330725795 y[1,1,28] + 238.12577943828234 y[1,1,29] + 246.87700413112805 y[1,1,30] + [[...20809 terms omitted...]] - s[11] - s[12] - s[13] - s[14] - s[15] - s[16] - s[17] - s[

In [22]:
optimize!(m)

Error: basis matrix is singular to working precision (cond = 1.17e+016)


In [23]:
solution_summary(m)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 5.75937e+11
  Objective bound    : 5.75937e+11
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.77340e+01


In [24]:
println(value.(x))

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.531412e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 884870.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3.45942371e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 7.989161374e6 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.060423101e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5.391570787e6 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5.61795e6 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 